# Analyzing Milwaukee Police Call Data and Weather Data
### Grant Fass and Chris Hubbell

## Introduction
Across the world, there are many crimes commited every hour. One of the greatest challenges is reducing crime and maintaining safety for citizens. Part of preventing crime relies on the reporting of it by citizens. If nobody informs the police, the police are unable to act. This is why reporting crimes and incidents is so important, especially when people's lives are in danger. In Wisconsin, Milwaukee Police Department (MPD) releases data regarding all of their dispatch calls, which we have been able to get since 2016. This allows for analyzing trends of crime reporting over time as well as as it relates to other factors. In 2010, Milwaukee installed a new system for detecting gun shots called ShotSpotter, which was expanded into more neighborhoods in 2014. This system is capable of detecting when a shot is fired and where it was to a high degree of accuracy. The data consists of both ShotSpotter calls as well as Shots Fired calls. The key difference is that Shots Fired are calls from people and ShotSpotter are automatic.

## Research Questions:
- Is there a significant difference between the distribution of shots spotted over time and calls for shots fired?
- Is there a significant difference in the proportion of calls that were unable to be located for shots fired calls compared to shots spotted?
- Does the Proportion of shots spotted and fired correlate with certain dates including holidays and events?
- Does the number of calls correlate with certain weather conditions?
- ~~Is it possible to predict number of calls based on location and district?~~
- Is it possible to predict the nature of a call based on its location and district?

## Hypotheses:
- There are significantly more shots spotted than calls about shots fired.
- Significantly more shots fired calls are unable to be located than shots spotted.
- There will be significantly more shots spotted calls on July 4th, Dec. 31st, and Jan 1st than normal days.
- There will be significantly less shots fired calls on holidays than normal days.
- There are significantly more calls on days with clear weather than inclement weather.
- There are significantly more calls on days around 75 degrees than there are on days around 95 or 55 degrees.
- ~~The number of calls will be able to be predicted based on location and district.~~
- The type of call will be unable to be predicted based on location and district.

## Notes:
One of the research questions and one of the hypotheses is striked out. This is because it was not possible to come up with a good method for measuring this in time.

# Imports
These are the libraries that will be relvant for working with the dataset.

In [ ]:
%matplotlib inline
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import kstest
from scipy.stats import ks_2samp
from scipy.stats import chi2_contingency
from IPython.display import Image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics import roc_curve, plot_roc_curve, precision_recall_curve, plot_precision_recall_curve

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

sns.set(rc={"figure.figsize":(12, 6)})

# Loading the Data
This section is used to load the data and make sure that all of the features have been formatted using the correct types. This data is ready for use since it has already been cleaned in another notebook. The MPDDataCleaning notebook was used to clean the MPD (Milwaukee Police Department) dataset. The WeatherDataCleaning notebook was used to clean the weather dataset. These two datasets were then combined in the DatasetCombining notebook.

In [ ]:
df = pd.read_csv('merged_data.csv')

In [ ]:
df.info()

## Revising Feature Types
Calling the [`.info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html) command shows that there are a number of features that are improperly formatted. The district, nature, status, primaryStreetName, primaryStreetSuffix, secondaryStreetName, secondaryStreetSuffix, and weatherDesc all need to become categorical features. The datetime feature needs to be changed to datetime.

In [ ]:
# df['district'] = df['district'].astype('category')
# df['nature'] = df['nature'].astype('category')
df['status'] = df['status'].astype('category')
df['primaryStreetName'] = df['primaryStreetName'].astype('category')
df['primaryStreetSuffix'] = df['primaryStreetSuffix'].astype('category')
df['secondaryStreetName'] = df['secondaryStreetName'].astype('category')
df['secondaryStreetSuffix'] = df['secondaryStreetSuffix'].astype('category')
df['weatherDesc'] = df['weatherDesc'].astype('category')
df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True)
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df['shots_nature'] = df['shots_nature'].astype('category')
df['top_natures'] = df['top_natures'].astype('category')
df['top_districts'] = df['top_districts'].astype('category')

## Examining The Loaded Data
The data should now be in the proper types. This will be examined using the [`.head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html), [`.info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html), and [`.describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) methods.

In [ ]:
df.head(5).T

In [ ]:
df.tail(5).T

In [ ]:
df.info(verbose=True, show_counts=True)

In [ ]:
df.describe()

## TODO: Explain the Loaded Data


# Examining the Research Questions
This section will seek to examine the research questions outlined at the top of the notebook by exploring the data through the use of both graphs and statistical tests. This section will use a few possible different statistical tests based on the scenario and shape of the data. The [Kolmogorov-Smirnov test](http://www.mit.edu/~6.s085/notes/lecture5.pdf) can be used to test if two arbitrary distributions are the same. [It does not require the data being normally distributed](http://statstutor.ac.uk/resources/steps-glossary/glossary/nonparametric.html#:~:text=The%20Kolmogorov%2DSmirnov%20test%20does,Squared%20Goodness%20of%20Fit%20Test.&text=The%20Kruskal%2DWallis%20test%20is,compare%20three%20or%20more%20samples.&text=It%20is%20the%20analogue%20to,used%20in%20analysis%20of%20variance.). The [two sample t-test](https://www.jmp.com/en_us/statistics-knowledge-portal/t-test/two-sample-t-test.html) can be used to test if the means of two distributions are the same. The [Kruskal-Wallis test](https://en.wikipedia.org/wiki/Kruskal%E2%80%93Wallis_one-way_analysis_of_variance) can be used to determine if two samples come from the same distribution. The Kruskal-Wallis test does require that the data be normally distributed though. The [chi-squared test](https://stats.oarc.ucla.edu/spss/whatstat/what-statistical-analysis-should-i-usestatistical-analyses-using-spss/#:~:text=A%20chi%2Dsquare%20test%20is,relationship%20between%20two%20categorical%20variables.) will be used to test if there is a relationship between two categorical variables. The research questions that seek to determine if prediction is possible will be examined in a later section on building machine learning models.

## Preparation
Creating the graphs and running the statistical tests in the following sections will be much easier if some features are added to the dataset. Some of the key features include extracting different values of the datetime feature such as day, week, month, and year. This will allow for the exploration of different granularity levels. Some of the graphs use help from [this](https://www.statology.org/seaborn-legend-outside/) for moving the legend outside of the graph, and from [this](https://stackoverflow.com/a/60679315) for plotting multiple categories (fix legend not showing). [This](https://stackoverflow.com/questions/9847213/how-do-i-get-the-day-of-week-given-a-date) stackoverflow post helped with extracting day of the week from datetime.

In [ ]:
# def nature_remap(n: str) -> str:
#     """
#     method to remap nature to be just 3 categories. SHOTSPOTTER, SHOTS FIRED, and OTHER
#     :param n: the nature to remap
#     :return: the remapped nature
#     """
#     if n not in ['SHOTSPOTTER', 'SHOTS FIRED']:
#         return 'OTHER'
#     else:
#         return n

In [ ]:
# def nature_remap_top_50(nature: str, unique_natures: list) -> str:
#     """
#     method to remap nature to be just 3 categories. SHOTSPOTTER, SHOTS FIRED, and OTHER
#     :param nature: the nature to remap
#     :return: the remapped nature
#     """
#     if nature not in unique_natures[0:50]:
#         return 'OTHER'
#     else:
#         return nature

In [ ]:
# def district_remap(value: str) -> str:
#     """
#     method to remap district to include the 7 Milwaukee police districts 
#     and one OTHER district.
#     :param value: the value of district to remap
#     :return: the remapped value of district
#     """
#     if value != value: # https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-values
#         return 'EMPTY'
#     if value not in ['1', '2', '3', '4', '5', '6', '7']:
#         return 'OTHER'
#     else:
#         return value

In [ ]:
# top_50_natures = list(df['nature'].value_counts().index)[0:50]
# df['minNature'] = df['nature'].map(nature_remap)
# df['minNature'] = df['minNature'].astype('category')
# df['top_50_natures'] = df.apply(lambda t: nature_remap_top_50(t['nature'], top_50_natures), axis=1)
# df['top_50_natures'] = df['top_50_natures'].astype('category')
# df['top_districts'] = df['district'].astype('object').map(district_remap)
# df['top_districts'] = df['top_districts'].astype('category')

In [ ]:
# df['date'] = df['datetime'].map(lambda t: t.date()) # Represents only the date with no time of day attached.
# df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True) # Change the type to a datetime (all the time values will be 0) this is so it can be graphed easier
# df['year'] = df['datetime'].map(lambda t: t.year) # Represents year
# df['month'] = df['datetime'].map(lambda t: t.month) # Represents month of the year
# df['week'] = df['datetime'].map(lambda t: t.week) # Represents week of the year
# df['day'] = df['datetime'].map(lambda t: t.day) # Represents day of the month
# df['hour'] = df['datetime'].map(lambda t: t.hour) # Represents hour of the day
# df['weekday'] = df['datetime'].map(lambda t: t.weekday()) # Monday is 0 and Sunday is 6

A filtered dataframe can be created containing only the entries for weapon crime. Extra categorical values must be removed when filtering down a categorical feature with many values (such as nature). This can be done by redefining the type as a category.

In [ ]:
shots_df = df[df['top_natures'].isin(['SHOTSPOTTER', 'SHOTS FIRED'])][['top_natures', 'status', 'datetime', 'date', 'year', 'month', 'week', 'day', 'hour', 'weekday']].copy(deep=True)
shots_df['top_natures'] = shots_df['top_natures'].astype('object').astype('category')
shots_df['notLocated'] = shots_df['status']=='Unable to Locate Complainant'
print("Data Shape After: %s" % ((shots_df.shape), ))
shots_df.head(10).T

## Research Question 1: Is There a Significant Difference Between the Distribution of Shots Spotted Over Time and Calls for Shots Fired?
The first step in answering this research question is to graph the distributions of shots spotted and shots fired over time. These can be compared over different possible time ranges. Some of the possible time ranges for comparisons include day of the week, day of the month, week of the year, month of the year, and daily over the entire time range of the dataset. These graphs can then be looked at to predict if a dataset is normally distributed or not. The outcome of this prediction will then determine what statistical test should be used.

### Graphing Shots Spotted vs. Shots Fired Over Time

In [ ]:
# plot of shots spotted vs shots fired over time
# Graph is not the best but is the most legible out of possible formats.
uses = shots_df['top_natures'].unique()
# plt.figure(figsize=(7,7))
ax = plt.axes()
for use in uses:
    sns.kdeplot(x=shots_df["date"], hue=shots_df[shots_df['top_natures']==use]["top_natures"], ax=ax, common_norm=False, multiple="layer", alpha=1, label=use)
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
ax.set_title("Shots Spotted vs. Shots Fired Calls Over Time")
ax.set_xlabel('Date')
plt.show()

In [ ]:
ax = sns.countplot(x="year", hue="top_natures", data=shots_df)
ax.set_xlabel('Year')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Shotspotter vs Shots Fired calls by Year')
plt.show()

In [ ]:
ax = sns.countplot(x="month", hue="top_natures", data=shots_df)
ax.set_xlabel('Month of the Year')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Shotspotter vs Shots Fired calls by Month of the Year')
plt.show()

In [ ]:
ax = sns.countplot(x="week", hue="top_natures", data=shots_df)
ax.set_xlabel('Week of the Year')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Shotspotter vs Shots Fired calls by Week of the Year')
plt.show()

This graph shows that there is some anomalous data since there are not 53 weeks in a year.

In [ ]:
ax = sns.countplot(x="day", hue="top_natures", data=shots_df)
ax.set_xlabel('Day of the Month')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Shotspotter vs Shots Fired calls by Day of the Month')
plt.show()

In [ ]:
ax = sns.countplot(x="hour", hue="top_natures", data=shots_df)
ax.set_xlabel('Hour of the Day')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Shotspotter vs Shots Fired calls by Hour of the Day')
plt.show()

### Performing Statistical Tests to Determine if the Distributions are Simmilar.
The above graphs show that neither the distribution of shots fired or shots spotted are normal for any time increment. This means that the Kruskal-Wallis statistical test cannot be used. The [Kolmogorov-Smirnov test, from the scipy stats library,](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html) will be used to determine if the number of shots spotted and shots called are simmilar. The [two sample version](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html#scipy.stats.ks_2samp) of the test will be used. The 'two sided' mode will be used for this test. This defines the null hypothesis to be that the two distributions are identical and the alternative to be that they are not identical.

In [ ]:
_, p = ks_2samp(shots_df[shots_df['top_natures'] == 'SHOTSPOTTER']['date'], shots_df[shots_df['top_natures'] == 'SHOTS FIRED']['date'], alternative='two-sided')
print('The p-value for the Kolmogorov-Smirnov test between the distributions of shots spotted and shots fired over time is: %e' % p)

### Conclusions for Research Question 1
#### TODO
run more tests?

## Research Question 2: Is There a Significant Difference in the Proportion of Calls That Were Unable to be Located for Shots Fired Calls Compared to Shots Spotted?

### Graphs

In [ ]:
ax = sns.heatmap(data=pd.crosstab(shots_df['top_natures'], shots_df['notLocated']), annot=True)
ax.set_xlabel('Unable to be Located')
ax.set_title('Calls Unable to be Located for Shots Spotted and Shots Fired')
plt.show()

### Performing Statistical Test
The [Chi-Squared test of independance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) is used to determine if there is a relationship between two or more variables by determining if they have a simmilar distribution. The [Chi-Squared test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html#scipy.stats.chisquare) is used to test if one feature has a specific distribution. This means the test of independance will be used.

In [ ]:
# combination_counts = original_cleaned_data[["type",target]].groupby(by=["type",target]).size().unstack(level=0).fillna(0)
# chi2, p, _, _ = stats.chi2_contingency(combination_counts)
compared = ['top_natures', 'notLocated']
_, p, _, _ = stats.chi2_contingency(shots_df[compared].groupby(by=compared).size().unstack(level=0).fillna(0))
print('The p-value for the test of independance between nature and notLocated is: %e' % p)

### Conclusions for Research Question 2
#### TODO

## Research Question 3: Does the Proportion of Shots Spotted and Shots Fired Correlate With Certain Dates Including Holidays and Events?
The following holdays will be used in this comparison:
- Independance day: July 4th, any year
- Christmas: December 25th, any year
- New Years Eve: December 31st, any year
- New Years: January 1st, any year
- Valentines Day: February 14th, any year
- Halloween: October 31st, any year
- Saint Patrick's Day: March 17th, any year

Thanksgiving will not be compared because it occurs on different days each year. Black friday will not be compared for the same reason. Due to the nature of this question it will likely be easiest to examine this question on a monthly basis instead of on a yearly basis.

### Graphs

In [ ]:
ax = sns.countplot(x="day", hue='top_natures', data=shots_df[shots_df['month'] == 7]) # 7 denotes the month of July
# ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
ax.set_xlabel('Day in July')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Calls Per Day in July')
plt.show()

This graph shows that there are a lot more calls for shots fired than normal on the fourth of July. This is likely due to people mistaking the sounds of fireworks in the distance for gunshots. This is likely reinforced by the shotspotter calls remaining much lower.

In [ ]:
ax = sns.countplot(x="day", hue='top_natures', data=shots_df[shots_df['month'] == 12]) # 12 denotes the month of December
# ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
ax.set_xlabel('Day in December')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Calls Per Day in December')
plt.show()

This graph shows that there is a large spike in shots fired and shots spotted on new years eve. It also shows that there may be a slight increase around Christmas. The slight increase around christmas may not be significant though.

In [ ]:
ax = sns.countplot(x="day", hue='top_natures', data=shots_df[shots_df['month'] == 1]) # 1 denotes the month of January
# ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
ax.set_xlabel('Day in January')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Calls Per Day in January')
plt.show()

This graph shows that there is a massive increase in calls for shots spotted and shots fired on new years day. There are more than four times as many calls on the first compared to other days of the month.

In [ ]:
ax = sns.countplot(x="day", hue='top_natures', data=shots_df[shots_df['month'] == 2]) # 2 denotes the month of February
# ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
ax.set_xlabel('Day in February')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Calls Per Day in February')
plt.show()

This graph shows that there may actually be a decrease in calls around valentines day compared to the rest of the month.

In [ ]:
ax = sns.countplot(x="day", hue='top_natures', data=shots_df[shots_df['month'] == 3]) # 3 denotes the month of March
# ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
ax.set_xlabel('Day in March')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Calls Per Day in March')
plt.show()

This graph is very jumpy which makes it hard to draw any conclusions from the graph alone. It does seem that there is not a significant difference for saint patrics day compared to other days.

In [ ]:
ax = sns.countplot(x="day", hue='top_natures', data=shots_df[shots_df['month'] == 10]) # 10 denotes the month of October
# ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
ax.set_xlabel('Day in October')
ax.set_ylabel('Number of Calls')
ax.set_title('Number of Calls Per Day in October')
plt.show()

This graph does not seem to show a large increase in shots spotted or shots fired on Halloween.

### Statistical Tests
#### TODO

## Research Question 4: Does the Number of Calls Correlate With Certain Weather Conditions?
<!-- could change to a heatmap where there are 3 columns. one for shot spotted, one for shot fired, and one for other. Compare the distribution of shot spotted vs other, compare shot fired vs other.  -->
### Graphs

In [ ]:
ax = sns.countplot(y="weatherDesc", data=df)
# ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
ax.set_xlabel('Number of Calls')
ax.set_ylabel('Weather Description')
ax.set_title('Number of Calls vs Weather Description')
plt.show()

### Statistical Tests
#### TODO

# Investigating the Shape of Data
This section looks to explore the shape of the data by generating graphs. These graphs will focus on the number of calls per some unit of time.

In [ ]:
ax = sns.histplot(data=df, x='date')
ax.set_title('All Calls Over Time')
ax.set_ylabel('Number of Calls')
ax.set_xlabel('Date')
plt.show()

This graph shows all of the call types over time. What is interesting is that there appears to be a big gap around 2018 and smaller gaps in 2019 and 2021. There is a second big gap between 2021 and 2022 but not as bad as the 2018 gap. Another interesting observation is that it appears that overall call numbers is trending down. Looking deeper:

In [ ]:
ax = sns.countplot(data=df, x='week')
ax.set_title('All Calls Over Week of the Year')
ax.set_ylabel('Number of Calls')
ax.set_xlabel('Week of the Year')
plt.show()

This graph shows a comparison between the number of calls recieved per week of the year. The number of calls looks to be consistent. There is one issue in that the data has 53 weeks each year. This should not be the case.

In [ ]:
ax = sns.countplot(data=df[(df['year'] == 2017) & (df['month']==2)], x='day')
ax.set_title('All Calls in February 2017')
ax.set_ylabel('Number of Calls')
ax.set_xlabel('Day of the Month')
plt.show()

In [ ]:
ax = sns.countplot(data=df[(df['year'] == 2017) & (df['month']==12)], x='day')
ax.set_title('All Calls in December 2017')
ax.set_ylabel('Number of Calls')
ax.set_xlabel('Day of the Month')
plt.show()

In [ ]:
ax = sns.countplot(data=df[(df['year'] == 2018) & (df['month']==1)], x='day')
ax.set_title('All Calls in January 2018')
ax.set_ylabel('Number of Calls')
ax.set_xlabel('Day of the Month')
plt.show()

In [ ]:
ax = sns.countplot(data=df[(df['year'] == 2021) & (df['month']==2)], x='day')
ax.set_title('All Calls in February 2021')
ax.set_ylabel('Number of Calls')
ax.set_xlabel('Day of the Month')
plt.show()

In [ ]:
ax = sns.countplot(data=df[(df['year'] == 2021) & (df['month']==8)], x='day')
ax.set_title('All Calls in August 2021')
ax.set_ylabel('Number of Calls')
ax.set_xlabel('Day of the Month')
plt.show()

In [ ]:
ax = sns.countplot(data=df[(df['year'] == 2021) & (df['month']==9)], x='day')
ax.set_title('All Calls in September 2021')
ax.set_ylabel('Number of Calls')
ax.set_xlabel('Day of the Month')
plt.show()

In [ ]:
ax = sns.countplot(data=df[(df['year'] == 2021) & (df['month']==10)], x='day')
ax.set_title('All Calls in October 2021')
ax.set_ylabel('Number of Calls')
ax.set_xlabel('Day of the Month')
plt.show()

## TODO
I emailed Nick to see if he can offer inisght here. At the moment, I'm guessing it's outage related - Chris
## TODO
Explain what each of the above graphs shows.

# Building Models to Test if Prediction is Possible
- models to test
    - Random Forest
    - SVM
    - Logistic Regression
- Graphs
    - Recall vs. Precision Graph
    - Confusion Matrix
    - ROC Curve

This section seeks to train models to answer the following research question:
- Is it possible to predict the nature of a call based on its location and district?

List all of the categorical features and numerical features

In [ ]:
df.info(verbose=True, show_counts=True)

In [ ]:
response_name = 'top_natures'

features = df.copy(deep=True)
# features = features.drop(columns=[response_name], axis=1)

response = df[response_name]

# https://stackoverflow.com/questions/16453644/regression-with-date-variable-using-scikit-learn
# drop date and datetime but keep the separated ones.
to_drop = ['date', 'datetime', 'weekday', 'shots_nature', 'traffic_crime', 'weapon_crime', 'isdaytime', response_name]

# features = features.drop(columns=['date', 'year', 'month', 'week', 'day', 'hour', 'weekday'])
numeric_features = ['call_id', 'houseNumber', 'loc_id', 'tempC', 'windspeedKmph', \
    'winddirdegree', 'precipMM', 'humidity', 'visibilityKm', 'pressureMB',\
        'cloudcover', 'HeatIndexC', 'DewPointC', 'WindChillC', 'WindGustKmph',\
            'FeelsLikeC', 'uvIndex', 'year', 'month', 'week', 'day', 'hour'] # , 'date', 'year', 'month', 'week', 'day', 'hour', 'weekday']

# features = features.drop(columns=['minNature', 'nature', 'traffic_crime', 
# 'weapon_crime', 'isdaytime',
# 'primaryStreetSuffix', 'secondaryStreetName', 'secondaryStreetSuffix', 'district'])

categorical_features = ['status', 'primaryStreetName', 'isCorner', \
    'primaryStreetSuffix', 'secondaryStreetName', 'secondaryStreetSuffix', 'top_districts']
features = features.drop(columns=to_drop)
# features = features.drop(columns=[response_name])

Next create the testing, training, and validation sets. The training and testing set sizes will be reduced since the overall number of data points exceedes four million.
<!-- Some help from [stackoverflow](https://stackoverflow.com/a/38251213) was used. This will create a 60%, 20%, 20% split. -->

In [ ]:
# train, validate, test = np.split(df.sample(frac=1, random_state=42), 
# [int(.6*len(df)), int(.8*len(df))])
train, test = train_test_split(df, test_size=0.005, train_size=0.1, stratify=df[response_name], random_state=42)

In [ ]:
# x_train = train.drop(columns=[response_name], axis=1)
# x_test = test.drop(columns=[response_name], axis=1)
# x_validate = validate.drop(columns=[response_name], axis=1)
x_train = train.drop(columns=to_drop, axis=1)
x_test = test.drop(columns=to_drop, axis=1)
# x_validate = validate.drop(columns=to_drop, axis=1)
y_train =train[response_name]
y_test = test[response_name]
# y_validate = validate[response_name]

Create the preprocessor pipeline for numeric and categorical features. This code comes from an in class example on the scat pipeline notebook presented during week 9.

In [ ]:
numeric_transformer = Pipeline(
    steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

Create different classifiers with and without PCA. Then define the grid search parameters and grid search sections. This code comes from the same in class example as the previous part.

In [ ]:
pipe_lr_svd = Pipeline([("preprocessor", preprocessor),
			('tSVD', TruncatedSVD(n_components=10)),
			('clf', LogisticRegression(random_state=42, solver='liblinear'))])

pipe_rf_svd = Pipeline([("preprocessor", preprocessor),
			('tSVD', TruncatedSVD(n_components=10)),
			('clf', RandomForestClassifier(random_state=42, n_jobs=-1))])
			
# pipe_svm_svd = Pipeline([("preprocessor", preprocessor),
# 			('tSVD', TruncatedSVD(n_components=10)),
# 			('clf', SVC(random_state=42))])

### Test the Different Models

In [ ]:
# List of pipelines for ease of iteration
grids = [pipe_lr_svd, pipe_rf_svd]

# Dictionary of pipelines and classifier types for ease of reference
grid_dict = {0: 'Logistic Regression w/tSVD', 1: 'Random Forest w/tSVD'}

In [ ]:
print('Performing model optimizations...')
best_acc = 0.0
best_clf = 0
best_gs = ''
for idx, gs in enumerate(grids):
	print('\nEstimator: %s' % grid_dict[idx])	
	# Fit grid search	
	gs.fit(x_train, y_train)
	# # Best params
	# print('Best params: %s' % gs.best_params_)
	# # Best training data accuracy
	# print('Best training accuracy: %.3f' % gs.best_score_)
	# # Predict on test data with best params
	y_pred = gs.predict(x_test)
	# Test data accuracy of model with best params
	print('Test set accuracy score for best params: %.3f ' % accuracy_score(y_test, y_pred))
	
	# Track best (highest test accuracy) model
	if accuracy_score(y_test, y_pred) > best_acc:
		best_acc = accuracy_score(y_test, y_pred)
		best_gs = gs
		best_clf = idx
print('\nClassifier with best test set accuracy: %s' % grid_dict[best_clf])

### Evaluating the Model
It looks like the Random Forest with TruncatedSVD was the best predictor out of the tested models. The next step is to evaluate how the model performed.

In [ ]:
y_pred = best_gs.predict(x_test)
cm = confusion_matrix(y_test, y_pred, labels=best_gs.classes_)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_gs.classes_)
disp.plot()
plt.show()

Unfortunatly, there are so many labels that the confusion matrix is somewhat illegible. This means test statistics will need to be relied upon.

In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
# from sklearn.metrics import roc_curve, plot_roc_curve, 
# precision_recall_curve, plot_precision_recall_curve
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
print('Random Forest w/tSVD Accuracy = %.3f, Precision = %.3f, Recall = %.3f' % (accuracy, precision, recall))

Based on the accuracy, precision, and recall of the model it appears like it is not possible to predict the nature of a call based on other factors about that call. 

# Conclusion
- Analysis of results, further study, improvements

# TODO Other:
- Explanations of all data and graph outputs
